In [1]:
from utils import load_env_vars
from openai import OpenAI
import time, json, os, tqdm

load_env_vars()
from utils_gpt import track_job, prepare_training_data

# Initialize OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [2]:
models = client.models.list()
for model in models:
    if "lamp" in model.id:
        print(model.id)

ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-p:AYKM4Mn4:ckpt-step-564
ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-p:AYKM5RNQ:ckpt-step-1128
ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-p:AYKM53Ac
ft:gpt-4o-mini-2024-07-18:salesforce-research:lamp-4o-mini-p-editor:AZpOfqCe:ckpt-step-1138
ft:gpt-4o-mini-2024-07-18:salesforce-research:lamp-4o-mini-p-editor:AZpOfQnM:ckpt-step-569
ft:gpt-4o-mini-2024-07-18:salesforce-research:lamp-4o-mini-p-editor:AZpOgz62
ft:gpt-4o-mini-2024-07-18:salesforce-research:lamp-4o-mini-s1000:AgGQZk3C:ckpt-step-500
ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-s1000:AgH71dow:ckpt-step-500
ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-s1000:AgH72s2W:ckpt-step-1000
ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-s1000:AgH72ocy
ft:gpt-4o-mini-2024-07-18:salesforce-research:lamp-4o-mini-s2000:AgcT9xmq:ckpt-step-500
ft:gpt-4o-mini-2024-07-18:salesforce-research:lamp-4o-mini-s2000:AgcTAsuK:ckpt-step-1000
ft:gpt-4o-mini-2024-07-18:salesforce-research

In [ ]:
# base_model, base_suffix = "gpt-4o-mini-2024-07-18", f"lamp-4o-mini"
base_model, base_suffix = "gpt-4o-2024-08-06", f"lamp-4o"

hyperparameters = {"n_epochs": 3}

tasks = [
    # {"suffix": "cot", "train_fn": "data/lamp_train_cot_format.json", "val_fn": "data/lamp_val_cot_format.json"},
    # {"suffix": "detection", "train_fn": "data/lamp_train_detection_format.json", "val_fn": "data/lamp_val_detection_format.json"},
    # {"suffix": "rewriting", "train_fn": "data/lamp_train_rewriting_format.json", "val_fn": "data/lamp_val_rewriting_format.json"},
    # {"suffix": "r_eval", "train_fn": "data/lamp_R_train.json", "val_fn": "data/lamp_R_val.json"},
    # {"suffix": "pr_eval", "train_fn": "data/lamp_PR_train.json", "val_fn": "data/lamp_PR_val.json"},
    {"suffix": "rsg2-eval", "train_fn": "data/lamp_Rsg2_train.json", "val_fn": "data/lamp_Rsg2_val.json"},
    {"suffix": "r-eval-b", "train_fn": "data/lamp_R_train.json", "val_fn": "data/lamp_R_val.json"},
    {"suffix": "p-eval-b", "train_fn": "data/lamp_P_train.json", "val_fn": "data/lamp_P_val.json"},

]
# ["S1000", "S2000", "S4000", "S8000"]

for task in tasks:
    suffix = f"{base_suffix}-{task['suffix']}"

    training_fn_gpt = f"data/tmp_gpt_tune_format_train.jsonl"
    prepare_training_data(task["train_fn"], training_fn_gpt)
    val_fn_gpt = f"data/tmp_gpt_tune_format_val.jsonl"
    prepare_training_data(task["val_fn"], val_fn_gpt)

    training_file = client.files.create(file=open(training_fn_gpt, 'rb'), purpose='fine-tune')
    val_file = client.files.create(file=open(val_fn_gpt, 'rb'), purpose='fine-tune')
    print(training_file, val_file)

    job = client.fine_tuning.jobs.create(training_file=training_file.id, model=base_model, suffix=suffix, validation_file=val_file.id, hyperparameters=hyperparameters)
    track_job(job.id)

FileObject(id='file-UALKmnJauGGQr6FmiAVpjd', bytes=3177061, created_at=1739243138, filename='tmp_gpt_tune_format_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None) FileObject(id='file-NWv1EAnh5NC7fChzDkBdxm', bytes=211663, created_at=1739243138, filename='tmp_gpt_tune_format_val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
[2025-02-10 22:05:41] Status: validating_files
